# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,23.78,75,4,1.78,ZA,1677225838
1,1,bathsheba,13.2167,-59.5167,24.34,73,20,6.17,BB,1677226081
2,2,bardiyah,31.7561,25.0865,14.76,75,0,3.56,LY,1677226081
3,3,mataura,-46.1927,168.8643,12.28,67,0,2.27,NZ,1677225818
4,4,hasaki,35.7333,140.8333,12.22,80,100,4.29,JP,1677225851


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.25,
    color = "City"
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df['Cloudiness'] == 0) & (city_data_df['Max Temp'] > 25.0),:]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,pakokku,21.3333,95.1000,33.18,24,0,1.24,MM,1677226085
97,97,hobart,-42.8794,147.3294,27.93,47,0,5.66,AU,1677225622
117,117,kavaratti,10.5669,72.6420,27.24,65,0,0.97,IN,1677225937
156,156,gopalpur,19.2667,84.9167,29.62,56,0,6.08,IN,1677226116
172,172,saint-pierre,-21.3393,55.4781,30.82,62,0,9.26,RE,1677225926
264,264,loikaw,19.6742,97.2094,32.71,12,0,1.21,MM,1677226142
294,294,manali,13.1667,80.2667,32.05,66,0,4.77,IN,1677226149
339,339,alibag,18.6411,72.8792,34.04,39,0,5.51,IN,1677226161
523,523,rajpipla,21.8667,73.5000,34.51,9,0,2.73,IN,1677226216
533,533,nayudupeta,13.9000,79.9000,32.49,27,0,4.35,IN,1677226221


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[['City','Country', 'Lat', 'Lng', 'Humidity']]
hotel_df = hotel_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,pakokku,MM,21.3333,95.1000,24,
97,hobart,AU,-42.8794,147.3294,47,
117,kavaratti,IN,10.5669,72.6420,65,
156,gopalpur,IN,19.2667,84.9167,56,
172,saint-pierre,RE,-21.3393,55.4781,62,
264,loikaw,MM,19.6742,97.2094,12,
294,manali,IN,13.1667,80.2667,66,
339,alibag,IN,18.6411,72.8792,39,
523,rajpipla,IN,21.8667,73.5000,9,
533,nayudupeta,IN,13.9000,79.9000,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "apiKey":geoapify_key,
    "limit":limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pakokku - nearest hotel: Shwe Pyi Hein
hobart - nearest hotel: Vibe Hotel Hobart
kavaratti - nearest hotel: No hotel found
gopalpur - nearest hotel: Swosti
saint-pierre - nearest hotel: Tropic Hotel
loikaw - nearest hotel: Nawaday Hotel
manali - nearest hotel: Vijay Park
alibag - nearest hotel: Hira Laxmi Residency
rajpipla - nearest hotel: No hotel found
nayudupeta - nearest hotel: Komala Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
25,pakokku,MM,21.3333,95.1000,24,Shwe Pyi Hein
97,hobart,AU,-42.8794,147.3294,47,Vibe Hotel Hobart
117,kavaratti,IN,10.5669,72.6420,65,No hotel found
156,gopalpur,IN,19.2667,84.9167,56,Swosti
172,saint-pierre,RE,-21.3393,55.4781,62,Tropic Hotel
264,loikaw,MM,19.6742,97.2094,12,Nawaday Hotel
294,manali,IN,13.1667,80.2667,66,Vijay Park
339,alibag,IN,18.6411,72.8792,39,Hira Laxmi Residency
523,rajpipla,IN,21.8667,73.5000,9,No hotel found
533,nayudupeta,IN,13.9000,79.9000,27,Komala Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [37]:
%%capture --no-display
# source: https://hvplot.holoviz.org/user_guide/Customization.html
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.25,
    color = "City",
    hover_cols = ['Hotel Name', 'Country']
)

map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)